# **차량 공유업체의 차량 파손 여부 분류하기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

dataset_path  = '/content/drive/MyDrive/Datasets/'

## 1.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

#### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
copy_train_path = dataset_path+'copy_images/trainset/'
copy_val_path = dataset_path+'copy_images/validset/'
copy_test_path = dataset_path+'copy_images/testset/'

# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

In [ ]:
train_x = []
val_x = []
test_x = []

for imgname in img_train_list:
    img = keras.utils.load_img(copy_train_path + imgname, color_mode='grayscale', target_size=(img_size, img_size))  # 이미지 로딩
    img = keras.utils.img_to_array(img)
    train_x.append(img)

for imgname in img_valid_list:
    img = keras.utils.load_img(copy_val_path + imgname, color_mode='grayscale', target_size=(img_size, img_size))  # 이미지 로딩
    img = keras.utils.img_to_array(img)
    val_x.append(img)

for imgname in img_test_list:
    img = keras.utils.load_img(copy_test_path + imgname, color_mode='grayscale', target_size=(img_size, img_size))  # 이미지 로딩
    img = keras.utils.img_to_array(img)
    test_x.append(img)

train_x = np.array(train_x)
val_x = np.array(val_x)
test_x = np.array(test_x)

print(type(train_x), len(train_x), train_x.shape)
print(type(val_x), len(val_x), val_x.shape)
print(type(test_x), len(test_x), test_x.shape)

In [ ]:
# min-max scaling
max_v, min_v = train_x.max(), train_x.min()

train_x = (train_x - min_v) / (max_v - min_v)
val_x = (val_x - min_v) / (max_v - min_v)
test_x = (test_x - min_v) / (max_v - min_v)

print('max :', train_x.max(),'  min :', train_x.min())

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
train_y = []
val_y = []
test_y = []

for imgname in img_train_list:
    if imgname.startswith('ab_') == True:
        train_y.append(1)
    else:
        train_y.append(0)
    
for imgname in img_valid_list:
    if imgname.startswith('ab_') == True:
        val_y.append(1)
    else:
        val_y.append(0)

for imgname in img_test_list:
    if imgname.startswith('ab_') == True:
        test_y.append(1)
    else:
        test_y.append(0)

In [ ]:
train_y = np.array(train_y)
val_y = np.array(val_y)
test_y = np.array(test_y)

print(train_y.shape, val_y.shape, test_y.shape)

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
train_x.shape, train_y.shape

In [ ]:
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 선언
model = keras.models.Sequential()

# 3. 모델 블록 조립
model.add(keras.layers.Input(shape=(280, 280, 1)))

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(keras.layers.Dropout(0.35))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1, activation='sigmoid'))

# 4. 컴파일
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist = model.fit(train_x, train_y,
                 validation_data=(val_x, val_y),
                 batch_size=64,
                 epochs=10000,
                 callbacks=[es],
                 verbose=1)

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
pred = model.predict(test_x)
y_pred = np.where(pred > 0.5, 1, 0)

print(confusion_matrix(test_y, y_pred))
print(classification_report(test_y, y_pred))

In [ ]:
if not isinstance(hist, dict):
    hist = hist.history

plt.subplot(1, 2, 1) 
plt.plot(hist['accuracy'])
plt.plot(hist['val_accuracy'])
plt.title('Accuracy : Training vs Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)

plt.subplot(1, 2, 2) 
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.title('Loss : Training vs Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)

plt.show()

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
train_path = dataset_path+'Car_Images_train/'
valid_path = dataset_path+'Car_Images_val/'
test_path = dataset_path+'Car_Images_test/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = ImageDataGenerator(
                    rotation_range=30,      
                    zoom_range = 0.3,       
                    width_shift_range=0.1,  
                    height_shift_range=0.1, 
                    horizontal_flip=True,   
                    vertical_flip=True,
                    rescale=1./255)     

valid_datagen = ImageDataGenerator(
                    rotation_range=30,      
                    zoom_range = 0.3,       
                    width_shift_range=0.1,  
                    height_shift_range=0.1, 
                    horizontal_flip=True,   
                    vertical_flip=True,
                    rescale=1./255)  

#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = train_datagen.flow_from_directory(train_path, 
                                                    target_size=(280, 280),
                                                    class_mode='binary',
                                                    batch_size=128,
                                                    shuffle=True)

valid_generator = valid_datagen.flow_from_directory(valid_path, 
                                                    target_size=(280, 280),
                                                    class_mode='binary',
                                                    batch_size=128,
                                                    shuffle=True)

#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

In [ ]:
keras.backend.clear_session()

il = keras.layers.Input(shape=(280, 280, 3))
hl = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')(il)
hl = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')(hl)
hl = keras.layers.BatchNormalization()(hl)
hl = keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(hl)

hl = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')(hl)
hl = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')(hl)
hl = keras.layers.BatchNormalization()(hl)
hl = keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(hl)

hl = keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')(hl)
hl = keras.layers.BatchNormalization()(hl)
hl = keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(hl)

hl = keras.layers.Flatten()(hl)
hl = keras.layers.Dense(256, activation='relu')(hl)
hl = keras.layers.Dense(256, activation='relu')(hl)
ol = keras.layers.Dense(1, activation='sigmoid')(hl)

model_da1 = keras.models.Model(il, ol)

model_da1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_da1.summary()

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist_da1 = model_da1.fit(train_generator,
                         validation_data=valid_generator,
                         batch_size=64,
                         epochs=10000,
                         callbacks=[es],
                         verbose=1)

#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
pred_da1 = model_da1.predict(test_x)
y_pred_da1 = np.where(pred_da1 > 0.005, 1, 0)

print(confusion_matrix(test_y, y_pred_da1))
print(classification_report(test_y, y_pred_da1))

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
keras.backend.clear_session()

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(280, 280, 3))
base_model.trainable=True

#### 2) VGG16과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

In [ ]:
new_output = keras.layers.Flatten()(base_model.output)
new_output = keras.layers.Dense(128, activation='relu')(new_output)
new_output = keras.layers.Dense(1, activation='sigmoid')(new_output)

model_tl1 = keras.models.Model(base_model.inputs, new_output)
model_tl1.summary()

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

In [ ]:
print(f'모델의 레이어 수 : {len(model_tl1.layers)}')

In [ ]:
for idx, layer in enumerate(model_tl1.layers) :
    if idx < 17 :
        layer.trainable = False  # Frozen(기존 가중치 사용) 학습하지 않음
    else :
        layer.trainable = True  # layer weight 조절. 학습.

In [ ]:
model_tl1.compile(loss='binary_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
# EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   min_delta=0, 
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist_tl1 = model_tl1.fit(train_x, train_y,
                         validation_data=(val_x, val_y),
                         batch_size=128,
                         epochs=10000,
                         callbacks=[es],
                         verbose=1)

#### 4) 성능 평가

In [ ]:
pred_tl1 = model_tl1.predict(test_x)
y_pred_tl1 = np.where(pred_tl1 > 0.5, 1, 0)
# y_pred_tl1 = pred_tl1.argmax(axis=1)

print(confusion_matrix(test_y, y_pred_tl1))
print(classification_report(test_y, y_pred_tl1))